# FLUX LoRA Training in Google Colab

이 노트북은 Google Colab에서 FLUX LoRA 훈련을 위한 환경설정 및 실행 가이드입니다.

In [ ]:
# GPU 정보 확인
!nvidia-smi
!nvcc --version

In [ ]:
# 필요한 패키지 설치
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers peft accelerate datasets
!pip install Pillow pandas numpy tqdm wandb safetensors
!pip install sentencepiece protobuf python-dotenv

In [ ]:
# Hugging Face 토큰 설정
import os
from google.colab import userdata

# Colab Secrets에서 토큰 가져오기 (권장)
try:
    HF_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    print("✅ Token loaded from Colab secrets")
except:
    # 직접 입력 (보안상 권장하지 않음)
    HF_TOKEN = "hf_your_token_here"  # 여기에 토큰 입력
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    print("⚠️ Token set manually")

In [ ]:
# 훈련 코드 다운로드
!git clone https://github.com/your-repo/flux-lora-training.git
# 또는 파일을 직접 업로드하세요

In [ ]:
# 데이터셋 업로드 또는 다운로드
# 방법 1: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 방법 2: 직접 업로드
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# GPU 메모리 최적화 설정
import torch

# GPU 메모리 확인
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    
    # 메모리 캐시 정리
    torch.cuda.empty_cache()

In [ ]:
# FLUX LoRA 훈련 실행
# T4 (16GB) 환경 최적화

!python train_lora.py \
    --data_dir "/content/drive/MyDrive/training_data" \
    --output_dir "./flux_lora_output" \
    --epochs 10 \
    --batch_size 1 \
    --lora_rank 8 \
    --learning_rate 5e-5